In [0]:
pip install -U openai markdown2

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from openai import OpenAI

In [0]:
#### PERFORM TEXT TRANSLATION
DATABRICKS_TOKEN = DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
def generate_report(review):
  # How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
  # DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
  # Alternatively in a Databricks notebook you can use this:
  client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="<link-to-the-databricks-model-serving-endpoint>"
  )

  chat_completion = client.chat.completions.create(
    messages=[
    {
      "role": "system",
      "content": "I sell cameras all around the world. My customers have left a bunch of feedback. Here is a summary of reviews. Generate a report that highlights all the major positive and negative feedback. Focus on the key complains and praises that my product team can use to improve the user experience. Your report should also be within 500 words. It should have 6 sections 'Title', 'Main Body', 'Positive Feedback', 'Negative Feedback', 'Recurring Complains', and 'Recommendations'."
    },
    {
      "role": "user",
      "content": f"{review}. Generate the report in markdown format."
    }
    ],
    model="databricks-meta-llama-3-1-70b-instruct",
    max_tokens=600
  )
  return chat_completion.choices[0].message.content

In [0]:
df_summary = spark.read.table("genai_demo.products.camera_reviews_with_sentiment_analysis").select("summary")
all_reviews = " ".join(filter(None, df_summary.rdd.map(lambda row: row.summary).collect()))

In [0]:
report = generate_report(all_reviews)

In [0]:
import markdown2
html = markdown2.markdown(report)
# Modify this to your own path
with open('/Volumes/genai_demo/products/reports/Product-Report-EN.html', 'w') as file:
    file.write(html)

In [0]:
# Save the content of the report in a markdown format - modify this path
with open('/Volumes/genai_demo/products/reports/Product-Report-Markdown.md', 'w') as file:
    file.write(report)